In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Funciones import *
# import warnings
# warnings.filterwarnings('ignore')

In [33]:
df = pd.read_csv("Football.csv")

df

,Country,League,home_team,away_team,home_score,away_score,season_year,Date_day,Date_hour,first_half,...,Distance_Covered_(km)_Home,Distance_Covered_(km)_Host,Clearances_Completed_Home,Clearances_Completed_Host,Pass_Success_per_Home,Pass_Success_per_Host,referee,venue,capacity,attendance
0,Germany,Bundesliga,B. Monchengladbach,Werder Bremen,4,1,2024/2025,3.11,19:30,3 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,Dankert B. (Ger),Borussia-Park (Mönchengladbach),54 042,54 042
1,Germany,Bundesliga,Freiburg,Mainz,0,0,2024/2025,3.11,17:30,0 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,Brand B. (Ger),Europa-Park Stadion (Freiburg),34 700,34 400
2,Germany,Bundesliga,Dortmund,RB Leipzig,2,1,2024/2025,2.11,20:30,1 - 1,...,NaN,NaN,NaN,NaN,NaN,NaN,Stieler T. (Ger),Signal Iduna Park (Dortmund),81 365,81 365
3,Germany,Bundesliga,Bayern Munich,Union Berlin,3,0,2024/2025,2.11,17:30,2 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,Jollenbeck M. (Ger),Allianz Arena (Munich),75 024,75 000
4,Germany,Bundesliga,Eintracht Frankfurt,Bochum,7,2,2024/2025,2.11,17:30,4 - 1,...,NaN,NaN,NaN,NaN,NaN,NaN,Schlager D. (Ger),Deutsche Bank Park (Frankfurt),58 000,58 000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95379,Spain,Primera-rfef-group-1,CF Talavera,Ath Bilbao B,4,0,2021/2022,20.03,19:00,3 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Estadio El Prado (Talavera de la Reina),4 200,NaN
95380,Spain,Primera-rfef-group-1,Extremadura UD,UD Logrones,0,2,2021/2022,20.03,19:00,0 - 2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Estadio Francisco de la Hera (Almendralejo),11 580,NaN
95381,Spain,Primera-rfef-group-1,Internacional de Madrid,CD Badajoz,1,0,2021/2022,20.03,14:00,0 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95382,Spain,Primera-rfef-group-1,Unionistas,Valladolid Promesas,0,0,2021/2022,20.03,14:00,0 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# Definir las ligas que forman parte del top 5 europeo
ligas_top5 = [
    "Premier League",
    "Bundesliga",
    "Serie A",
    "Ligue 1",
    "La Liga"
]

# Crear nuevo DataFrame con solo esas ligas
df_top5 = df[df["League"].isin(ligas_top5)].copy()


In [35]:
df_top5

,Country,League,home_team,away_team,home_score,away_score,season_year,Date_day,Date_hour,first_half,...,Distance_Covered_(km)_Home,Distance_Covered_(km)_Host,Clearances_Completed_Home,Clearances_Completed_Host,Pass_Success_per_Home,Pass_Success_per_Host,referee,venue,capacity,attendance
0,Germany,Bundesliga,B. Monchengladbach,Werder Bremen,4,1,2024/2025,3.11,19:30,3 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,Dankert B. (Ger),Borussia-Park (Mönchengladbach),54 042,54 042
1,Germany,Bundesliga,Freiburg,Mainz,0,0,2024/2025,3.11,17:30,0 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,Brand B. (Ger),Europa-Park Stadion (Freiburg),34 700,34 400
2,Germany,Bundesliga,Dortmund,RB Leipzig,2,1,2024/2025,2.11,20:30,1 - 1,...,NaN,NaN,NaN,NaN,NaN,NaN,Stieler T. (Ger),Signal Iduna Park (Dortmund),81 365,81 365
3,Germany,Bundesliga,Bayern Munich,Union Berlin,3,0,2024/2025,2.11,17:30,2 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,Jollenbeck M. (Ger),Allianz Arena (Munich),75 024,75 000
4,Germany,Bundesliga,Eintracht Frankfurt,Bochum,7,2,2024/2025,2.11,17:30,4 - 1,...,NaN,NaN,NaN,NaN,NaN,NaN,Schlager D. (Ger),Deutsche Bank Park (Frankfurt),58 000,58 000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7299,Germany,Bundesliga,Freiburg,Stuttgart,4,0,2000/2001,12.08,16:30,2 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dreisamstadion (Freiburg),24 000,NaN
7300,Germany,Bundesliga,Hamburger SV,Munich 1860,2,2,2000/2001,12.08,16:30,2 - 2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Volksparkstadion (Hamburg),57 274,NaN
7301,Germany,Bundesliga,Kaiserslautern,Bochum,0,1,2000/2001,12.08,16:30,0 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fritz Walter Stadion (Kaiserslautern),49 500,NaN
7302,Germany,Bundesliga,Werder Bremen,Cottbus,3,1,2000/2001,12.08,16:30,2 - 1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Weserstadion (Bremen),42 100,NaN


In [40]:
df_top5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7304 entries, 0 to 7303
Data columns (total 41 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Country                               7304 non-null   object 
 1   League                                7304 non-null   object 
 2   home_team                             7304 non-null   object 
 3   away_team                             7304 non-null   object 
 4   home_score                            7304 non-null   object 
 5   away_score                            7304 non-null   object 
 6   season_year                           7304 non-null   object 
 7   first_half                            7296 non-null   object 
 8   second_half                           7294 non-null   object 
 9   home_team_goals_current_time          3616 non-null   object 
 10  home_team_goals                       3146 non-null   object 
 11  away_team_goals_curren

In [37]:
df_top5.describe()

,Date_day,expected_goals_xg_home,expected_goals_xg_host,Goal_Attempts_Home,Goal_Attempts_Host,Shots_on_Goal_Home,Shots_on_Goal_Host,Shots_off_Goal_Home,Shots_off_Goal_Host,Blocked_Shots_Home,...,Attacks_Home,Attacks_Host,Dangerous_Attacks_Home,Dangerous_Attacks_Host,Distance_Covered_(km)_Home,Distance_Covered_(km)_Host,Clearances_Completed_Home,Clearances_Completed_Host,Pass_Success_per_Home,Pass_Success_per_Host
count,7304.000000,529.000000,529.000000,3476.000000,3476.000000,3476.000000,3476.000000,3476.000000,3476.000000,1663.000000,...,2186.000000,2186.000000,2186.000000,2186.000000,261.000000,261.000000,0.0,0.0,2.000000,2.000000
mean,15.851809,1.825425,1.425633,14.164269,11.718354,5.610472,4.579114,6.997411,5.848101,3.253157,...,110.398445,101.887466,49.904849,42.706313,118.072797,117.808429,NaN,NaN,81.500000,85.000000
std,8.620885,1.039741,0.862755,5.274759,4.842476,3.118410,2.680881,3.526629,3.269559,2.200047,...,28.504301,27.162991,19.056705,17.025589,5.033586,4.932787,NaN,NaN,0.707107,2.828427
min,1.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,104.000000,104.000000,NaN,NaN,81.000000,83.000000
25%,8.120000,1.060000,0.810000,10.000000,8.000000,3.000000,3.000000,4.000000,3.000000,2.000000,...,92.000000,85.000000,37.000000,31.000000,115.000000,115.000000,NaN,NaN,81.250000,84.000000
50%,16.090000,1.660000,1.280000,14.000000,11.000000,5.000000,4.000000,7.000000,5.000000,3.000000,...,110.000000,101.000000,47.000000,40.000000,118.000000,118.000000,NaN,NaN,81.500000,85.000000
75%,23.040000,2.460000,1.890000,17.000000,15.000000,7.000000,6.000000,9.000000,8.000000,5.000000,...,127.750000,118.000000,61.000000,52.000000,121.000000,121.000000,NaN,NaN,81.750000,86.000000
max,31.100000,6.590000,4.550000,36.000000,32.000000,22.000000,20.000000,24.000000,21.000000,13.000000,...,215.000000,207.000000,145.000000,112.000000,151.000000,151.000000,NaN,NaN,82.000000,87.000000


In [38]:
df_top5.columns = df_top5.columns.str.replace("host", "away")
df_top5.columns = df_top5.columns.str.replace("Host", "Away")

In [39]:
# Lista de columnas que quiero borrar
columnas_a_borrar = [
    "referee", "venue", "attendance", "capacity", "Blocked_Shots_Away", "Blocked_Shots_Home", "away_team_yellow_card_current_time", "home_team_goals_assist",
    "Free_Kicks_Home", "Shots_off_Goal_Away","away_team_red_card_current_time","away_team_red_card", "home_team_red_card", "Distance_Covered_(km)_Away",
    "Free_Kicks_Away", "Shots_off_Goal_Home","home_team_yellow_card_current_time", "Clearances_Completed_Home", "Clearances_Completed_Away", "Interceptions_Home",
    "Crosses_Completed_Home", "Crosses_Completed_Away", "Free_Kicks_Away","away_team_substitution_why","home_team_red_card_current_time", "Interceptions_Away",
    "Throw_ins_Away", "Throw_ins_Home", "Corner_Kicks_Away", "home_team_substitution_why","away_team_goals_assist", "Distance_Covered_(km)_Home",
    "home_team_goals_current_score", "away_team_goals_current_score", "Date_hour", "Date_day", "Tackles_Away", "Tackles_Home", "Pass_Success_per_Away", "Pass_Success_per_Away",
    "Corner_Kicks_Home", "Pass_Success_per_Home", "Dangerous_Attacks_Home", "Dangerous_Attacks_Away", "Attacks_Home", "Attacks_Away", 
    "Completed_Passes_Home", "Completed_Passes_Away", "home_team_yellow_card", "away_team_yellow_card", "Goal_Attempts_Home", "Goal_Attempts_Away"
]

# Borrarlas del Dataset
df_top5= df_top5.drop(columns=columnas_a_borrar)

In [9]:
df_top5

,Country,League,home_team,away_team,home_score,away_score,season_year,first_half,second_half,home_team_goals_current_time,...,Goalkeeper_Saves_Home,Goalkeeper_Saves_Away,Fouls_Home,Fouls_Away,Red_Cards_Home,Red_Cards_Away,Yellow_Cards_Home,Yellow_Cards_Away,Total_Passes_Home,Total_Passes_Away
0,Germany,Bundesliga,B. Monchengladbach,Werder Bremen,4,1,2024/2025,3 - 0,1 - 1,"[""11'"", ""12'"", ""45'"", ""66'""]",...,4.0,4.0,12.0,10.0,0.0,1.0,1.0,4.0,479.0,549.0
1,Germany,Bundesliga,Freiburg,Mainz,0,0,2024/2025,0 - 0,0 - 0,NaN,...,2.0,2.0,9.0,12.0,NaN,NaN,1.0,2.0,471.0,443.0
2,Germany,Bundesliga,Dortmund,RB Leipzig,2,1,2024/2025,1 - 1,1 - 0,"[""30'"", ""65'""]",...,2.0,5.0,11.0,12.0,NaN,NaN,3.0,4.0,528.0,418.0
3,Germany,Bundesliga,Bayern Munich,Union Berlin,3,0,2024/2025,2 - 0,1 - 0,"[""15'"", ""43'"", ""51'""]",...,1.0,4.0,2.0,9.0,NaN,NaN,0.0,2.0,866.0,244.0
4,Germany,Bundesliga,Eintracht Frankfurt,Bochum,7,2,2024/2025,4 - 1,3 - 1,"[""9'"", ""18'"", ""20'"", ""32'"", ""61'"", ""66'"", ""69'""]",...,6.0,2.0,6.0,15.0,NaN,NaN,1.0,2.0,594.0,324.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7299,Germany,Bundesliga,Freiburg,Stuttgart,4,0,2000/2001,2 - 0,2 - 0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7300,Germany,Bundesliga,Hamburger SV,Munich 1860,2,2,2000/2001,2 - 2,0 - 0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7301,Germany,Bundesliga,Kaiserslautern,Bochum,0,1,2000/2001,0 - 0,0 - 1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7302,Germany,Bundesliga,Werder Bremen,Cottbus,3,1,2000/2001,2 - 1,1 - 0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
columnas_rellenables = [
    "Red_Cards_Home", "Red_Cards_Away",
    "Yellow_Cards_Home", "Yellow_Cards_Away",
    "Fouls_Home", "Fouls_Away",
    "Offsides_Home", "Offsides_Away"
]

df_top5[columnas_rellenables] = df_top5[columnas_rellenables].fillna(0)

df_top5[columnas_rellenables].isnull().sum()

Red_Cards_Home       0
Red_Cards_Away       0
Yellow_Cards_Home    0
Yellow_Cards_Away    0
Fouls_Home           0
Fouls_Away           0
Offsides_Home        0
Offsides_Away        0
dtype: int64

In [48]:
df_top5["resultado"] = df_top5.apply(lambda row: "local" if row["home_score"] > row["away_score"] 
                           else "visitante" if row["home_score"] < row["away_score"] 
                           else "empate", axis=1)
df_top5

,Country,League,home_team,away_team,home_score,away_score,season_year,first_half,second_half,home_team_goals_current_time,...,Fouls_Away,Red_Cards_Home,Red_Cards_Away,Yellow_Cards_Home,Yellow_Cards_Away,Total_Passes_Home,Total_Passes_Away,resultado,diferencia_goles,Mayor_Posesion
0,Germany,Bundesliga,B. Monchengladbach,Werder Bremen,4,1,2024/2025,3 - 0,1 - 1,"[""11'"", ""12'"", ""45'"", ""66'""]",...,10.0,0.0,1.0,1.0,4.0,479.0,549.0,local,3,visitante
1,Germany,Bundesliga,Freiburg,Mainz,0,0,2024/2025,0 - 0,0 - 0,NaN,...,12.0,0.0,0.0,1.0,2.0,471.0,443.0,empate,0,local
2,Germany,Bundesliga,Dortmund,RB Leipzig,2,1,2024/2025,1 - 1,1 - 0,"[""30'"", ""65'""]",...,12.0,0.0,0.0,3.0,4.0,528.0,418.0,local,1,local
3,Germany,Bundesliga,Bayern Munich,Union Berlin,3,0,2024/2025,2 - 0,1 - 0,"[""15'"", ""43'"", ""51'""]",...,9.0,0.0,0.0,0.0,2.0,866.0,244.0,local,3,local
4,Germany,Bundesliga,Eintracht Frankfurt,Bochum,7,2,2024/2025,4 - 1,3 - 1,"[""9'"", ""18'"", ""20'"", ""32'"", ""61'"", ""66'"", ""69'""]",...,15.0,0.0,0.0,1.0,2.0,594.0,324.0,local,5,local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7299,Germany,Bundesliga,Freiburg,Stuttgart,4,0,2000/2001,2 - 0,2 - 0,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,local,4,NaN
7300,Germany,Bundesliga,Hamburger SV,Munich 1860,2,2,2000/2001,2 - 2,0 - 0,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,empate,0,NaN
7301,Germany,Bundesliga,Kaiserslautern,Bochum,0,1,2000/2001,0 - 0,0 - 1,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,visitante,1,NaN
7302,Germany,Bundesliga,Werder Bremen,Cottbus,3,1,2000/2001,2 - 1,1 - 0,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,local,2,NaN


In [43]:
# Asegurar que las columnas de goles estén en formato entero

df_top5["home_score"] = pd.to_numeric(df_top5["home_score"]).astype(int)

df_top5["away_score"] = pd.to_numeric(df_top5["away_score"]).astype(int)

In [29]:
# Eliminar % y convertir a float
df_top5["Ball_Possession_Home"] = df_top5["Ball_Possession_Home"].astype(str).str.replace("%", "").astype(float)
df_top5["Ball_Possession_Away"] = df_top5["Ball_Possession_Away"].astype(str).str.replace("%", "").astype(float)

In [44]:
# Eliminar filas donde home_score o away_score sean el string "-"

# Crear máscara booleana para esas filas
condicion = (df_top5["home_score"] == "-") & (df_top5["away_score"] == "-")

# Usar drop con los índices que cumplen la condición
df_top5 = df_top5.drop(index=df_top5[condicion].index)

In [45]:
# Diferencia de goles
df_top5["diferencia_goles"] = df_top5["home_score"] - df_top5["away_score"]

# Cambiamos la columna por su valor absoluto para tener la diferencia siempre en positivo
df_top5["diferencia_goles"] = df_top5["diferencia_goles"].abs()

df_top5

,Country,League,home_team,away_team,home_score,away_score,season_year,first_half,second_half,home_team_goals_current_time,...,Fouls_Home,Fouls_Away,Red_Cards_Home,Red_Cards_Away,Yellow_Cards_Home,Yellow_Cards_Away,Total_Passes_Home,Total_Passes_Away,resultado,diferencia_goles
0,Germany,Bundesliga,B. Monchengladbach,Werder Bremen,4,1,2024/2025,3 - 0,1 - 1,"[""11'"", ""12'"", ""45'"", ""66'""]",...,12.0,10.0,0.0,1.0,1.0,4.0,479.0,549.0,local,3
1,Germany,Bundesliga,Freiburg,Mainz,0,0,2024/2025,0 - 0,0 - 0,NaN,...,9.0,12.0,0.0,0.0,1.0,2.0,471.0,443.0,empate,0
2,Germany,Bundesliga,Dortmund,RB Leipzig,2,1,2024/2025,1 - 1,1 - 0,"[""30'"", ""65'""]",...,11.0,12.0,0.0,0.0,3.0,4.0,528.0,418.0,local,1
3,Germany,Bundesliga,Bayern Munich,Union Berlin,3,0,2024/2025,2 - 0,1 - 0,"[""15'"", ""43'"", ""51'""]",...,2.0,9.0,0.0,0.0,0.0,2.0,866.0,244.0,local,3
4,Germany,Bundesliga,Eintracht Frankfurt,Bochum,7,2,2024/2025,4 - 1,3 - 1,"[""9'"", ""18'"", ""20'"", ""32'"", ""61'"", ""66'"", ""69'""]",...,6.0,15.0,0.0,0.0,1.0,2.0,594.0,324.0,local,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7299,Germany,Bundesliga,Freiburg,Stuttgart,4,0,2000/2001,2 - 0,2 - 0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,local,4
7300,Germany,Bundesliga,Hamburger SV,Munich 1860,2,2,2000/2001,2 - 2,0 - 0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,empate,0
7301,Germany,Bundesliga,Kaiserslautern,Bochum,0,1,2000/2001,0 - 0,0 - 1,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,visitante,1
7302,Germany,Bundesliga,Werder Bremen,Cottbus,3,1,2000/2001,2 - 1,1 - 0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,local,2


In [46]:
# Crear columna: quién tuvo más posesión, o NaN si falta alguno

df_top5["Mayor_Posesion"] = df_top5.apply(
    lambda row: (
        "local" if row["Ball_Possession_Home"] > row["Ball_Possession_Away"]
        else "visitante" if row["Ball_Possession_Home"] < row["Ball_Possession_Away"]
        else "empate" if pd.notnull(row["Ball_Possession_Home"]) and pd.notnull(row["Ball_Possession_Away"])
        else np.nan
    ),
    axis=1
)


In [47]:
df_top5


,Country,League,home_team,away_team,home_score,away_score,season_year,first_half,second_half,home_team_goals_current_time,...,Fouls_Away,Red_Cards_Home,Red_Cards_Away,Yellow_Cards_Home,Yellow_Cards_Away,Total_Passes_Home,Total_Passes_Away,resultado,diferencia_goles,Mayor_Posesion
0,Germany,Bundesliga,B. Monchengladbach,Werder Bremen,4,1,2024/2025,3 - 0,1 - 1,"[""11'"", ""12'"", ""45'"", ""66'""]",...,10.0,0.0,1.0,1.0,4.0,479.0,549.0,local,3,visitante
1,Germany,Bundesliga,Freiburg,Mainz,0,0,2024/2025,0 - 0,0 - 0,NaN,...,12.0,0.0,0.0,1.0,2.0,471.0,443.0,empate,0,local
2,Germany,Bundesliga,Dortmund,RB Leipzig,2,1,2024/2025,1 - 1,1 - 0,"[""30'"", ""65'""]",...,12.0,0.0,0.0,3.0,4.0,528.0,418.0,local,1,local
3,Germany,Bundesliga,Bayern Munich,Union Berlin,3,0,2024/2025,2 - 0,1 - 0,"[""15'"", ""43'"", ""51'""]",...,9.0,0.0,0.0,0.0,2.0,866.0,244.0,local,3,local
4,Germany,Bundesliga,Eintracht Frankfurt,Bochum,7,2,2024/2025,4 - 1,3 - 1,"[""9'"", ""18'"", ""20'"", ""32'"", ""61'"", ""66'"", ""69'""]",...,15.0,0.0,0.0,1.0,2.0,594.0,324.0,local,5,local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7299,Germany,Bundesliga,Freiburg,Stuttgart,4,0,2000/2001,2 - 0,2 - 0,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,local,4,NaN
7300,Germany,Bundesliga,Hamburger SV,Munich 1860,2,2,2000/2001,2 - 2,0 - 0,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,empate,0,NaN
7301,Germany,Bundesliga,Kaiserslautern,Bochum,0,1,2000/2001,0 - 0,0 - 1,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,visitante,1,NaN
7302,Germany,Bundesliga,Werder Bremen,Cottbus,3,1,2000/2001,2 - 1,1 - 0,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,local,2,NaN
